# Bow river above Banff
In this exercise you will run different setups of the Structure for Unifying Multiple Modelling Alternatives (SUMMA; Clark et al., 2015a,b) for part of the Bow river in Alberta, Canada. Each setup is a different way in which the modelling domain can be discretized. You will investigate the impact of the choice of spatial discretization on streamflow simulations for this region. The map below gives an impression of the size and elevation gradiants in this catchment. The red circle indicates the catchment's outlet. (Sub-)catchment boundaries are taken from the MERIT Hydro dataset (Yamazaki et al., 2019). The Bow Valley contains several glaciers that strongly influence the regions streamflow generation. See Bash and Marshall (2014) for an introduction to glacier melt contributions in the Bow valley and Naz et al. (2014) for an example of glacier modelling.

<div>
<img src="./img/bow_at_banff.png" width="750"/>
</div>

### Note on pysumma
In the following, you might want to change certain values in your pysumma setup. Keep in mind that if you make these changes in the input files in the `settings` folder, you will need to make a new pysumma simulation object to have these changes take effect (`s = ps.Simulation(executable, file_manager_constant)`). If you instead make the changes to an existing pysumma simulation object, make sure to reset your pysumma simulation object to its base state after each simulation (`s.reset()`) so that your changes don't accumulate. 

## References
Bash, E., & Marshall, S. (2014). Estimation of glacial melt contributions to the Bow River, Alberta, Canada, using a radiation-temperature melt model. Annals of Glaciology, 55(66), 138-152. doi:10.3189/2014AoG66A226

Clark, M. P., J. Hendrikx, A. G. Slater, D. Kavetski, B. Anderson, N. J. Cullen, T. Kerr, E. Orn Hreinsson, and R. A. Woods (2011), Representing spatial variability of snow water equivalent in hydrologic and land-surface models: A review, Water Resour. Res., 47, W07539, doi:10.1029/2011WR010745

Clark, M. P., Nijssen, B., Lundquist, J. D., Kavetski, D., Rupp, D. E., Woods, R. A., … Rasmussen, R. M. (2015a). A unified approach for process-based hydrologic modeling: 1. Modeling concept. Water Resources Research, 51(4), 2498–2514. https://doi.org/10.1002/2015WR017198

Clark, M. P., Nijssen, B., Lundquist, J. D., Kavetski, D., Rupp, D. E., Woods, R. A., … Marks, D. G. (2015b). A unified approach for process-based hydrologic modeling: 2. Model implementation and case studies. Water Resources Research, 51, 2515–2542. https://doi.org/10.1002/2015WR017200

Naz, B. S., Frans, C. D., Clarke, G. K. C., Burns, P., and Lettenmaier, D. P. (2014). Modeling the effect of glacier recession on streamflow response using a coupled glacio-hydrological model, Hydrol. Earth Syst. Sci., 18, 787–802, https://doi.org/10.5194/hess-18-787-2014

Mizukami, N., Clark, M. P., Sampson, K., Nijssen, B., Mao, Y., McMillan, H., Viger, R. J., Markstrom, S. L., Hay, L. E., Woods, R., Arnold, J. R., and Brekke, L. D. (2016). mizuRoute version 1: a river network routing tool for a continental domain water resources applications, Geosci. Model Dev., 9, 2223–2238, https://doi.org/10.5194/gmd-9-2223-2016

Yamazaki, D., Ikeshima, D., Sosa, J., Bates, P. D., Allen, G. H., & Pavelsky, T. M. (2019). MERIT Hydro: a high‐resolution global hydrography map based on latest topography dataset. Water Resources Research, 55, 5053–5073. https://doi.org/10.1029/2019WR024873

<br>

## Notebook setup

In [ ]:
# modules
import os
import time
import subprocess # needed to run mizuRoute.exe from the notebook
import numpy as np
import xarray as xr
import pysumma as ps
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
# define the base path so that we won't have to type this over and over
path_base = '/home/jovyan/data/pbhmCourse_student/2_process_based_modelling/'

<br>

## Load the simulations from Hydroshare
Simulations for this exercise have already been prepared and are available as a Hydroshare resource. Code to run SUMMA and perform the routing is available at the end of this notebook for those who are interested to try this out themselves. Note that the distributed simulations can take over an hour to complete, which is why these simulations are already prepared for your use. Execute the following commands to load the simulations from Hydroshare.

In [ ]:
# modules
from hs_restclient import HydroShare # to get the files
import zipfile # and to unzip them

In [ ]:
# Make a hydroshare object - note: needs authentication
hs = HydroShare()

In [ ]:
# Specify the resource ID and download the resource into the main exercise directory
resource_id = 'a0cfaa6df18643ccbf777c9e3cdd8a86'
hs.getResource(resource_id, 
               destination=path_base)

In [ ]:
# unzip the file 
with zipfile.ZipFile(resource_id+'.zip', 'r') as zip_ref:
    
    # find all files in the zip
    zip_files = zip_ref.namelist()
    
    # loop over the files and extract only the .nc
    for zip_file in zip_files:
        if zip_file.endswith('.nc'):
            
            # print
            print('Now extracting ' + zip_file)
            
            # extract the file
            zip_ref.extract(zip_file,'tmp_zip')
            
            # find where the file is inside tmp_zip
            for root, dirs, files in os.walk('./tmp_zip'):
                for file in files:
                    old_loc = os.path.join(root,file)
                    old_folder = root
                    
            # specify where it should go
            new_folder = '.' + old_folder[old_folder.find('/output/'):]
            new_loc = '.' + old_loc[old_loc.find('/output/'):]
            
            # make the output directory if it doesn't exist yet
            if not os.path.exists(new_folder):
                os.makedirs(new_folder)
            
            # Take the file out of the temporary folder and put it where it shoule go
            os.rename(old_loc,new_loc)

In [ ]:
# remove the file and the tmp_zip dir
!rm -r 'a0cfaa6df18643ccbf777c9e3cdd8a86.zip'
!rm -r './tmp_zip/'

<br>

## Spatial discretization
Now that all results are in, let's take take a look at the different modelling domains that generated the results. The four different spatial discretization setups are:
- The catchment is treated as a single, lumped entity. In SUMMA terms, it consists of a single Geographical Response Unit (GRU)
- The catchment is treated as a single GRU, but subdivided into three Hydrological Response Units (HRUs) that represent different elevation bands
- The cachment is divided into 51 GRUs that represent different sub-catchments, based on the river network
- The catchment is divided into 51 sub-catchments (GRUs), where the GRUs are further subdivided into HRUs that represent different elevation bands

We'll vizualize this using the shapefiles (`.shp`) provided as part of the exercise data.

In [ ]:
# Specify shape file locations
path_shp_lumped           = path_base + 'data/bow/topography/lumped/bow_lumped.shp'
path_shp_lumped_elev      = path_base + 'data/bow/topography/lumped_elevationBands/bow_lumped_elevation_zone.shp'
path_shp_distributed      = path_base + 'data/bow/topography/distributed/bow_distributed.shp'
path_shp_distributed_elev = path_base + 'data/bow/topography/distributed_elevationBands/bow_distributed_elevation_zone.shp'
path_shp_river_network    = path_base + 'data/bow/topography/river_network/bow_river_network_from_merit_hydro.shp'

In [ ]:
# Load the shapes
shp_lumped      = gpd.read_file(path_shp_lumped)
shp_lumped_elev = gpd.read_file(path_shp_lumped_elev)
shp_dist        = gpd.read_file(path_shp_distributed)
shp_dist_elev   = gpd.read_file(path_shp_distributed_elev)
shp_network     = gpd.read_file(path_shp_river_network)

In [ ]:
# make a helpful function that can be used to plot the data
def plot_cases(basin_shape,ax,title_string):
    
    vmin, vmax = 1380,3320 # [m]
    basin_shape.plot(ax=ax, column='hru_elev', facecolor='w', edgecolor='k', cmap='Greys_r', legend=True, vmin=vmin, vmax=vmax); # basin discretization
    shp_network.plot(ax=ax, linewidth=2, color=(66/255,203/255,245/255)); # river network (the same for all cases)
    ax.plot(-115.555,51.167,linestyle='None',marker='o',color='r',label='outlet') # outlet (the same for all cases)
    ax.legend()
    ax.set_title(title_string)
    
    for im in plt.gca().get_images():
        im.set_clim(vmin, vmax)
    
    return

In [ ]:
# Prepare a figure
fig, ax = plt.subplots(2,2,figsize=(14,14));
fig.tight_layout()
fig.suptitle('Mean HRU elevation [m.a.s.l.] for modelling setups of Bow river above Banff')
fig.subplots_adjust(top=0.92, wspace=0.27, hspace=0.15) # move stuff around a bit

# plot the 4 cases
plot_cases(shp_lumped,     ax[0,0],'Case 1: lumped catchment \n1 GRU')
plot_cases(shp_lumped_elev,ax[0,1],'Case 2: lumped catchment with elevation bands \n 1 GRU containing 3 HRUs')
plot_cases(shp_dist,       ax[1,0],'Case 3: distributed sub-catchments \n 51 GRUs')
plot_cases(shp_dist_elev,  ax[1,1],'Case 4: distributed sub-catchments with elevation bands \n 51 GRUs with up to 3 HRUs')

<br>

## Model setup
Now we'll need to setup a pysumma simulation object. You won't need to run the models for this exercise (although you can with the code at the end of this notebook if you wnat to), but the psyumma objects can be useful to access the parameter and attribute values for each model setup.

In [ ]:
# SUMMA.exe location
executable = '/usr/local/bin/summa.exe'

In [ ]:
# Define location of .exe and file manager
file_manager_lumped           = path_base + 'settings/bow/lumped/fileManager.txt'
file_manager_lumped_elev      = path_base + 'settings/bow/lumped_elevationBands/fileManager.txt'
file_manager_distributed      = path_base + 'settings/bow/distributed/fileManager.txt'
file_manager_distributed_elev = path_base + 'settings/bow/distributed_elevationBands/fileManager.txt'

In [ ]:
# Open pysumma simulations for both
s_lumped           = ps.Simulation(executable, file_manager_lumped)
s_lumped_elev      = ps.Simulation(executable, file_manager_lumped_elev)
s_distributed      = ps.Simulation(executable, file_manager_distributed)
s_distributed_elev = ps.Simulation(executable, file_manager_distributed_elev)

<br>

## Assignment - subjective decisions in spatial discretization of model domains
For this assignment you will run SUMMA through its pysumma interface for the four different modelling setups presented above. For the distributed cases you will also run mizuRoute (Mizukam et al., 2016) to do flow routing between the 51 GRUs. Internal routing for elevation zones (in SUMMA terms, each elevation zone is an HRU within the GRU that defines the (sub) catchment) is handled by SUMMA.

<br>
<br>

## Exercise 1: snow analysis
The spatial discretization of models affects the magnitude, timing and duration of the simulated snow pack and thus the magnitude, timing and duration of simulated streamflow. One way to visualize this is to plot the evolution of modelled Snow Water Equivalent (SWE) for each of the modelling elements in the four model setups (see Fig. 9 in Clark et al., 2011). SWE can be found in the SUMMA output `.nc` files as variable `scalarSWE`. As an example, code is provided that creates such a plot for cases 1 (lumped) and 2 (lumped with elevation bands). For this part of the exercise:
- Create a similar plot for the two distributed model setups
- Calculate a few metrics for each of the SWE simulations that quantify certain statistical properties of the simulation that you think are useful to understand the differences between these simulations. You can for example calculate a metric that quantifies the amount of SWE across the whole basin (magnitude) or a metric that quantifies the amount of time needed before all SWE has melted (timing). Clark et al. (2011) can give you further ideas.
- Comment on the differences

#### SWE plot for the two lumped cases

In [ ]:
# define the output paths
# -- SUMMA results
path_sim_lumped            = path_base + 'output/bow/lumped/bow_lumped_default_timestep.nc'
path_sim_lumped_elev       = path_base + 'output/bow/lumped_elevationBands/bow_lumped_elev_default_timestep.nc'
path_sim_distr             = path_base + 'output/bow/distributed/bow_distributed_default_timestep.nc'
path_sim_distr_elev        = path_base + 'output/bow/distributed_elevationBands/bow_distributed_elev_default_timestep.nc'

# -- mizuRoute results (contains streamflow only)
path_sim_distr_routed      = path_base + 'output/bow/distributed/bow_distributed_routed.nc'
path_sim_distr_elev_routed = path_base + 'output/bow/distributed_elevationBands/bow_distributed_elev_routed.nc'

In [ ]:
# load the SUMMA simulations, we won't need the routing results for SWE analysis
sim_lumped      = xr.open_dataset(path_sim_lumped)
sim_lumped_elev = xr.open_dataset(path_sim_lumped_elev)
sim_distr       = xr.open_dataset(path_sim_distr)
sim_distr_elev  = xr.open_dataset(path_sim_distr_elev)

In [ ]:
# make the plot
plt.figure(figsize=(30,7));

sim_lumped.scalarSWE.plot(color='r', label='Case 1: lumped', linewidth=3);
sim_lumped_elev.scalarSWE.plot.line(x='time', color='grey'); # not plotting labels for these helps with memory usage
sim_lumped_elev.scalarSWE.mean(dim='hru').plot(color='b', label='Case 2: average of elevation bands (individual bands in grey)', linewidth=3)

plt.legend(loc='upper left'); # forcing the legend in a specific location helps with memory usage
plt.ylabel('simulated SWE [kg m-2]');

In [10]:
import xarray as xr

<br>
<br>

## Exercise 2: streamflow analysis
The flow regime in the Bow river is predominantly melt-driven, and as such the spatial discretization of the model that affects magnitude, timing and duration of SWE simulations also affects the magnitude, timing and duration of streamflow simulations. Simulated streamflow for the four model setups can be accessed as follows:
- Case 1, lumped: as variable `averageRoutedRunoff` in the model output `.nc` file, with units `[m s-1]`
- Case 2, lumped + elevation zones: as variable `averageRoutedRunoff` in the model output `.nc` file, with units `[m s-1]`
- Case 3: distributed: as variable `IRFroutedRunoff` in the routing output `.nc` file, with units `[m3 s-1]`
- Case 4: distributed + elevation zones: as variable `IRFroutedRunoff` in the routing output `.nc` file, with units `[m3 s-1]`

Due to the differences in spatial discretization, some further details about these streamflow simulations are needed:
- Case 1, lumped: this single runoff time series is assumed to be from the basin outlet (and is for all intents and purposes identical to that in `scalarRunoffTotal`).
- Case 2, lumped + elevation zones: time series of runoff for each of the three HRUs are assumed to flow into the unresolved river network (i.e. the river network smaller than the resolution for which we use mizuRoute for routing) and are routed internally with a Unit Hydrograph approach. This results in a single runoff time series for the GRU (i.e. the combination of the three HRUs).
- Case 3, distributed: the routing model mizuRoute provides routed streamflow for each reach in the river network. The reach at the outlet has `reachID = 71028585`.
- Case 4, distributed + elevation zones: see Case 2 and 3. Within-GRU routing is done within SUMMA; routing between GRUs is handled by mizuRoute.


For this exercise:
- Make sure the units of the simulations match by convert the streamflow series from either the lumped or the distributed cases. The catchment area that you need for this can be found as part of the local attributes of each pysumma instantiation (e.g. `s_lumped.local_attributes.HRUarea`).
- Create one or more figures to show the differences in streamflow simulations. Examples could be the full time series for all four model instantions, the average annual hydrograph for each of the four cases (i.e. the average value of all 1-Jan 00:00's, 1-Jan 01:00's, etc), etc.
- Calculate a few metrics for each of the streamflow simulations that quantify certain statistical properties of the simulation that you think are useful to understand the differences between these simulations. The metrics you calcualted for SWE could be useful for streamflow analysis too.
- Comment on the simulation differences

<br>

## Exercise 3: synthesis
Reflect on your findings about the impact of spatial discretization in models in the snow and streamflow simulations the model generates. Questions to consider:
- How does the spatial discretization of the catchment affect catchment-average simulations of snow and streamflow?
- How could the model setup be further adjusted to closer approach physical reality? 

<br>
<br>

***
## Extra: code to run these models for yourself
In case you're curious about model run times for different setups or wish to do some analysis that invovles changing model parametrizations or parameters, we here provide the code needed to run each of the four SUMMA setups. We've also included the code needed to do the routing of the two distributed setups. Note that these model setups are run in series. Parallelization is the focus of a later lecture. Enjoy!

In [ ]:
# Make the output directory if it doesn't exist
if not os.path.exists(s_lumped.manager['outputPath'].value):
    os.makedirs(s_lumped.manager['outputPath'].value)
    
if not os.path.exists(s_lumped_elev.manager['outputPath'].value):
    os.makedirs(s_lumped_elev.manager['outputPath'].value)    
    
if not os.path.exists(s_distributed.manager['outputPath'].value):
    os.makedirs(s_distributed.manager['outputPath'].value) 
    
if not os.path.exists(s_distributed_elev.manager['outputPath'].value):
    os.makedirs(s_distributed_elev.manager['outputPath'].value) 

<br>

#### Case 1: lumped catchment
In the lumped case, the catchment is treated as a single (lumped) entity. In SUMMA terms, this model setup uses a single GRU (Geographic Response Unit). No routing is needed as a result.

In [ ]:
# Initialize a timer
case1_time_start = time.time()

# Run the lumped model with default settings
s_lumped.run('local', run_suffix='default')
print(s_lumped.status)

# End the time and show result
case1_time_end = time.time()
print('Model run took ' + str(round(case1_time_end - case1_time_start)) + ' sec.')

<br>

#### Case 2: lumped catchment with elevation bands
This setup introduces some spatial complexity, by defining elevation zones within the catchment. In SUMMA terms, the single lumped GRU (see case 1) is now sub-divided into three HRUs (Hydrological Response Units). SUMMA internally handles the routing from the highest HRU to the lowest. No external routing is needed.

In [ ]:
# Initialize a timer
case2_time_start = time.time()

# Run the model with the default settings
s_lumped_elev.run('local', run_suffix='default')
print(s_lumped_elev.status)

# End the time and show result
case2_time_end = time.time()
print('Model run took ' + str(round(case2_time_end - case2_time_start)) + ' sec.')

<br>

#### Case 3: distributed catchment
This setup introduces a different way of spatial discretizaiton, namely sub-dividing a larger catchment into multiple sub-catchments. This setup uses the MERIT Hydro basin delineation (Yamazaki et al., 2019) which provides 51 sub-catchments for our modelling domain. In SUMMA terms, this model setup uses 51 GRUs (technically, the setup also includes 51 HRUs that exactly overlap the GRUs - each GRU is "divided" into 1 HRU). SUMMA does not provide GRU-to-GRU routing and external routing is needed.

**SUMMA**

In [ ]:
# Initialize a timer
case3_time_summa_start = time.time()

# Run the model with the default settings
s_distributed.run('local', run_suffix='default')
print(s_distributed.status)

# End the time and show result
case3_time_summa_end = time.time()
print('Model run took ' + str(round(case3_time_summa_end - case3_time_summa_start)) + ' sec.')

**mizuRoute**

In [ ]:
# paths to routing .exe and settings
mizuExe = 'route_runoff.exe'
mizuArg = path_base + 'mizuRoute/settings/bow_distributed.control'

In [ ]:
# Initialize a timer
case3_time_mizu_start = time.time()

# run mizuRoute through subprocess module
log = subprocess.run([mizuExe, mizuArg], capture_output=True)

# End the time and show result
case3_time_mizu_end = time.time()
print('Model run took ' + str(round(case3_time_mizu_end - case3_time_mizu_start)) + ' sec.')

**mizuRoute post-processing**

In [ ]:
# routing post-processing: find the individual annual mizuRoute output files
files = []
for file in os.listdir(s_distributed.manager['outputPath'].value):
    if 'mizuRoute' in file:
        files.append(s_distributed.manager['outputPath'].value + '/' + file)

# make sure they are in the right order
files.sort() 

In [ ]:
# routing post-processing: merge the annual files into a single file
save_file = s_distributed.manager['outputPath'].value + '/' + s_distributed.manager['outFilePrefix'].value + '_routed.nc' # new file name
ds = xr.merge([xr.open_dataset(file) for file in files]) # merge individual files
ds.to_netcdf(save_file) # save to file

<br>

#### Case 4: distributed catchment with elevation bands
This setup introduces even more spatial complexity, by sub-dividing each of the 51 GRUs in up to three elevation zones. As a result each GRU can now consist of up to 3 HRUs. As before (see case 2), SUMMA handles the HRU-to-HRU routing, but external routing is needed to do the GRU-to-GRU routing and gives us simulated streamflow at the outlet.

**SUMMA**

In [ ]:
# Initialize a timer
case4_time_summa_start = time.time()

# Run the model with the default settings
s_distributed_elev.run('local', run_suffix='default')
print(s_distributed_elev.status)


# End the time and show result
case4_time_summa_end = time.time()
print('Model run took ' + str(round(case4_time_summa_end - case4_time_summa_start)) + ' sec.')

**mizuRoute**

In [ ]:
# paths to routing .exe and settings
mizuExe = 'route_runoff.exe'
mizuArg = path_base + 'mizuRoute/settings/bow_distributed_elevationBands.control'

In [ ]:
# Initialize a timer
case4_time_mizu_start = time.time()

# run mizuRoute.exe through subprocess module
log = subprocess.run([mizuExe, mizuArg], capture_output=True)

# End the time and show result
case4_time_mizu_end = time.time()
print('Model run took ' + str(round(case4_time_mizu_end - case4_time_mizu_start)) + ' sec.')

**mizuRoute postprocessing**

In [ ]:
# routing post-processing: find the individual annual mizuRoute output files
files = []
for file in os.listdir(s_distributed_elev.manager['outputPath'].value):
    if 'mizuRoute' in file:
        files.append(s_distributed_elev.manager['outputPath'].value + '/' + file)

# make sure they are in the right order
files.sort() 

In [ ]:
# routing post-processing: merge the annual files into a single one and remove the individual files
save_file = s_distributed_elev.manager['outputPath'].value + '/' + s_distributed_elev.manager['outFilePrefix'].value + '_routed.nc' # new file name
ds = xr.merge([xr.open_dataset(file) for file in files]) # merge individual files
ds.to_netcdf(save_file) # save to file

<br>

Now we can generate a plot of the computational trade-offs. Approximately linear, as might be expected.

In [ ]:
time_taken = [118, 207, 2513+867, 5273+846] # ugly, but lost the variables. Oh well..
comp_units = [1,   3,   51,       118]

In [ ]:
plt.plot(comp_units,time_taken,linestyle='None',marker='o');
plt.xlabel('number of computational units [-]');
plt.ylabel('computational time [s]');
plt.title('Computational cost of SUMMA and mizuRoute');